In [41]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import cross_val_score, KFold

from scipy.stats import sem

import warnings
warnings.simplefilter("ignore")

In [5]:
news = fetch_20newsgroups(subset='all')

In [10]:
type(news.data)

list

In [22]:
news.target

array([10,  3, 17, ...,  3,  1,  7])

In [18]:
print(news.target_names)
print('\n')
type(news.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']




list

In [19]:
news.data[0]

"From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>\nSubject: Pens fans reactions\nOrganization: Post Office, Carnegie Mellon, Pittsburgh, PA\nLines: 12\nNNTP-Posting-Host: po4.andrew.cmu.edu\n\n\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n"

In [21]:
news.target_names[0]

'alt.atheism'

In [23]:
Split_perc = 0.75
split_size = int(len(news.data)*Split_perc)

In [25]:
x_train = news.data[:split_size]
x_test = news.data[split_size:]
y_train = news.target[:split_size]
y_test = news.target[split_size:]

In [35]:
print('x_test : ',len(x_test))
print('x_train : ', len(x_train))
print('y_test : ',len(y_test))
print('y_train : ', len(y_train))

x_test :  4712
x_train :  14134
y_test :  4712
y_train :  14134


In [38]:
clf_1 = Pipeline([('vect', CountVectorizer()),
                 ('clf', MultinomialNB())])

In [39]:
clf_2 = Pipeline([('vect', HashingVectorizer()),
                 ('clf', MultinomialNB())])

In [40]:
clf_3 = Pipeline([('vect', TfidfVectorizer()),
                 ('clf', MultinomialNB())])